In [1]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model, evaluate, find_adv_k, sentiment
import model_adv
from bertviz import head_view
import torch

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
(train_dataset, test_dataset), info = get_datasets(batch_size=64)
encoder = info.features['text'].encoder
vocab_size=info.features['text'].encoder.vocab_size 

In [4]:
transformer, _optimizer, _checkpoint, _manager = create_model(models_dir="models/pos_enc_True", load_checkpoint=True, 
                           vocab_size=vocab_size, use_positional_encoding=True, run_eagerly=True)

# adv = model_adv.create_model(models_dir="adv", load_checkpoint=False, run_eagerly=True, d_model=512)

Loaded previously trained model.


In [5]:
# Initial accuracy
# metrics = evaluate(transformer, test_dataset)
# print(metrics.result().numpy())

In [6]:
adv_model, adv_optimizer = model_adv.create_model(models_dir="adv", load_checkpoint=False, 
                                            run_eagerly=True, d_model=128)


In [9]:
x, y = next(iter(test_dataset))


sentence = x[0]
sentence_str = encoder.decode(x[0])
print(sentence_str)
tokens = [encoder.decode([w]) for w in sentence]

I've watched the movie actually several times. And what i want to say about it is the only thing that made this movie high rank was the Burak Altay's incredible performance, absolutely nothing but that. Not even those silly model named Deniz Akkaya and some of these popular names at times in the movie... Burak is definitely very talented i've seen a few jobs he made and been through. Even though this is kind of horror movie, he's doing really good job in comedy movies and also in dramas too. I bet most of you all saw Asmali Konak the movie and TV series, those two would go for an example... All i'm gonna say is you better watch out for the new works coming out from Burak then you'll see.. Keep the good work bro, much love..


In [39]:
#     for batch, (x, y) in enumerate(train_dataset):
loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
                adv_model, adv_optimizer, epochs=250)

Returned k: [  1   4 147 128]
[0] Loss_t = 0.008571237326      
[0] Loss_t (custom k) = 2.530784845         
[1 ] Loss_t = 3.503428459           Loss_k = -19.7434845            Loss = -8.120027542        
[2 ] Loss_t = 2.595817089           Loss_k = -19.76286697           Loss = -8.583524704        
[3 ] Loss_t = 1.033212543           Loss_k = -19.77846527           Loss = -9.372626305        
[4 ] Loss_t = 0.07953776419         Loss_k = -19.7920742            Loss = -9.856267929        
[5 ] Loss_t = 0.005254026968        Loss_k = -19.80577469           Loss = -9.900259972        
[6 ] Loss_t = 0.0008753265138       Loss_k = -19.81963921           Loss = -9.909381866        
[7 ] Loss_t = 0.0002890135802       Loss_k = -19.83363724           Loss = -9.91667366         
[8 ] Loss_t = 0.0001402318012       Loss_k = -19.84774208           Loss = -9.923800468        
[9 ] Loss_t = 8.392085147e-05       Loss_k = -19.86192513           Loss = -9.930920601        
[10] Loss_t = 5.730354678e-

[88] Loss_t = 7.900956916e-06       Loss_k = -20.96041679           Loss = -10.48020458        
[89] Loss_t = 7.898846889e-06       Loss_k = -20.97322083           Loss = -10.4866066         
[90] Loss_t = 7.897619071e-06       Loss_k = -20.98598289           Loss = -10.49298763        
[91] Loss_t = 7.896722309e-06       Loss_k = -20.99870682           Loss = -10.49934959        
[92] Loss_t = 7.896293937e-06       Loss_k = -21.01138878           Loss = -10.50569057        
[93] Loss_t = 7.896964235e-06       Loss_k = -21.02402878           Loss = -10.51201057        
[94] Loss_t = 7.89880869e-06        Loss_k = -21.03662872           Loss = -10.51831055        
[95] Loss_t = 7.900790479e-06       Loss_k = -21.04918671           Loss = -10.52458954        
[96] Loss_t = 7.903458936e-06       Loss_k = -21.06170464           Loss = -10.5308485         
[97] Loss_t = 7.906271094e-06       Loss_k = -21.0741806            Loss = -10.53708649        
[98] Loss_t = 7.909558917e-06       Loss

[173] Loss_t = 8.907660231e-06       Loss_k = -21.90626526           Loss = -10.95312786        
[174] Loss_t = 8.926019291e-06       Loss_k = -21.91583443           Loss = -10.95791245        
[175] Loss_t = 8.944485671e-06       Loss_k = -21.92537498           Loss = -10.96268272        
[176] Loss_t = 8.962887478e-06       Loss_k = -21.93488312           Loss = -10.96743679        
[177] Loss_t = 8.981574865e-06       Loss_k = -21.94436073           Loss = -10.9721756         
[178] Loss_t = 9.000360478e-06       Loss_k = -21.95380974           Loss = -10.9769001         
[179] Loss_t = 9.019092431e-06       Loss_k = -21.96322632           Loss = -10.98160839        
[180] Loss_t = 9.03772434e-06        Loss_k = -21.97261238           Loss = -10.98630142        
[181] Loss_t = 9.056525414e-06       Loss_k = -21.98196983           Loss = -10.99098015        
[182] Loss_t = 9.075364687e-06       Loss_k = -21.99129677           Loss = -10.99564362        
[183] Loss_t = 9.094269444e-06

In [40]:
# sentence = "This is a very terrible movie . I will never watch this bad movie again ."
# sentence = "this was a perfect movie . I enjoyed it a lot"
# tokens = sentence.split()

sent_original, _ = sentiment(sentence_str, encoder, transformer)
sent, weights = sentiment(sentence_str,    encoder=encoder, 
                      adv_model=adv_model, transformer=transformer)
# sent, weights = sentiment(sentence, encoder=encoder, transformer=old_transformer)


print(f"Initial sentiment: {sent_original}")
print(f'Adv sentiment: {sent}')

Initial sentiment: neg
Adv sentiment: pos


In [ ]:
head_view(torch.tensor(weights.numpy()[0]), tokens)

## Non Random K

In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=False)

In [8]:
# Adv after 1 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.836199


In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=False)

In [11]:
# Adv after 2 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.8323017


In [12]:
# Original accuracy
metrics = evaluate(transformer, test_dataset)
print(metrics.result().numpy())

0.82691425


## Random K

In [7]:
# Original accuracy
metrics = evaluate(transformer, test_dataset)
print(metrics.result().numpy())

0.82691425


In [8]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

Returned k: [ 64   4 197 128]
[0] Loss_t = 0.1774389148        
[0] Loss_t (custom k) = 0.7005707622        
[1 ] Loss_t = 0.7084273696          Loss_k = -10.93822956           Loss = -5.114901066        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.05808017403       
[0] Loss_t (custom k) = 0.3017933965        
[1 ] Loss_t = 0.2976107001          Loss_k = -11.18264103           Loss = -5.442515373        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1451408267        
[0] Loss_t (custom k) = 0.6754778624        
[1 ] Loss_t = 0.6705783606          Loss_k = -11.41095638           Loss = -5.37018919         
Returned k: [ 64   4 198 128]
[0] Loss_t = 0.1734466553        
[0] Loss_t (custom k) = 0.4421346486        
[1 ] Loss_t = 0.4331523776          Loss_k = -11.64333057           Loss = -5.605089188        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1056292132        
[0] Loss_t (custom k) = 0.5531953573        
[1 ] Loss_t = 0.5336462259          Loss_k = -11.88695431           Los

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.2065683901        
[0] Loss_t (custom k) = 0.7015267611        
[1 ] Loss_t = 0.7842805386          Loss_k = -17.34478951           Loss = -8.280254364        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1607328653        
[0] Loss_t (custom k) = 0.3670778573        
[1 ] Loss_t = 0.4614821076          Loss_k = -17.42427826           Loss = -8.481397629        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1055716649        
[0] Loss_t (custom k) = 0.4240380526        
[1 ] Loss_t = 0.4299423397          Loss_k = -17.49653244           Loss = -8.533294678        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2565942407        
[0] Loss_t (custom k) = 0.4721184969        
[1 ] Loss_t = 0.5219145417          Loss_k = -17.56642342           Loss = -8.52225399         
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1290135384        
[0] Loss_t (custom k) = 0.5135444403        
[1 ] Loss_t = 0.6424516439          Loss_k = -17.63356781           Los

Returned k: [ 64   4 196 128]
[0] Loss_t = 0.172894448         
[0] Loss_t (custom k) = 0.6312788725        
[1 ] Loss_t = 0.7494764328          Loss_k = -19.19112015           Loss = -9.220821381        
Returned k: [ 64   4 198 128]
[0] Loss_t = 0.05861240625       
[0] Loss_t (custom k) = 0.3671630025        
[1 ] Loss_t = 0.6480053663          Loss_k = -19.23138809           Loss = -9.29169178         
Returned k: [ 64   4 197 128]
[0] Loss_t = 0.2637432516        
[0] Loss_t (custom k) = 0.6691877246        
[1 ] Loss_t = 1.075423956           Loss_k = -19.25662994           Loss = -9.090602875        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1685515642        
[0] Loss_t (custom k) = 0.4850499928        
[1 ] Loss_t = 0.8111251593          Loss_k = -19.30076599           Loss = -9.244820595        
Returned k: [ 64   4 198 128]
[0] Loss_t = 0.1471689641        
[0] Loss_t (custom k) = 0.5721107721        
[1 ] Loss_t = 0.7413324714          Loss_k = -19.31958961           Los

Returned k: [ 64   4 198 128]
[0] Loss_t = 0.2358064055        
[0] Loss_t (custom k) = 0.5557926893        
[1 ] Loss_t = 0.9225178957          Loss_k = -20.07972145           Loss = -9.578601837        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1174945682        
[0] Loss_t (custom k) = 0.5597546697        
[1 ] Loss_t = 0.6995316148          Loss_k = -20.10227013           Loss = -9.701369286        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1413992345        
[0] Loss_t (custom k) = 0.4136630893        
[1 ] Loss_t = 0.4522666037          Loss_k = -20.12488556           Loss = -9.836309433        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1562387943        
[0] Loss_t (custom k) = 0.5306173563        
[1 ] Loss_t = 1.13879168            Loss_k = -20.14261818           Loss = -9.501913071        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.119096905         
[0] Loss_t (custom k) = 0.666859746         
[1 ] Loss_t = 1.052268863           Loss_k = -20.15925217           Los

In [9]:
# Adv after 1 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.5600642


In [10]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1640149951        
[0] Loss_t (custom k) = 0.5557336211        
[1 ] Loss_t = 0.9408072829          Loss_k = -20.3147583            Loss = -9.686975479        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1257707179        
[0] Loss_t (custom k) = 0.5690784454        
[1 ] Loss_t = 0.9774026871          Loss_k = -20.32094002           Loss = -9.671768188        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.09122480452       
[0] Loss_t (custom k) = 0.7012672424        
[1 ] Loss_t = 1.727092266           Loss_k = -20.33860207           Loss = -9.305754662        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.2567504048        
[0] Loss_t (custom k) = 0.6154232025        
[1 ] Loss_t = 1.22608161            Loss_k = -20.34514809           Loss = -9.559533119        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2060005069        
[0] Loss_t (custom k) = 0.4840722382        
[1 ] Loss_t = 1.09490633            Loss_k = -20.36059952           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.08303955942       
[0] Loss_t (custom k) = 0.4523341656        
[1 ] Loss_t = 1.166257381           Loss_k = -20.7611084            Loss = -9.79742527         
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1947315782        
[0] Loss_t (custom k) = 0.7699803114        
[1 ] Loss_t = 1.784237742           Loss_k = -20.77093315           Loss = -9.493348122        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2220232934        
[0] Loss_t (custom k) = 0.6712129116        
[1 ] Loss_t = 1.633943558           Loss_k = -20.78093529           Loss = -9.573495865        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.09362614155       
[0] Loss_t (custom k) = 0.4171029329        
[1 ] Loss_t = 1.114024401           Loss_k = -20.78964424           Loss = -9.837809563        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1907646805        
[0] Loss_t (custom k) = 0.6436902285        
[1 ] Loss_t = 1.23191905            Loss_k = -20.79426575           Los

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.3101727664        
[0] Loss_t (custom k) = 0.7166326046        
[1 ] Loss_t = 1.419758201           Loss_k = -21.12186241           Loss = -9.851052284        
Returned k: [ 64   4 192 128]
[0] Loss_t = 0.1678408682        
[0] Loss_t (custom k) = 0.5661774278        
[1 ] Loss_t = 1.007651448           Loss_k = -21.09453011           Loss = -10.04343891        
Returned k: [ 64   4 198 128]
[0] Loss_t = 0.2106332928        
[0] Loss_t (custom k) = 0.6585566998        
[1 ] Loss_t = 1.042074442           Loss_k = -21.13360786           Loss = -10.04576683        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1712105125        
[0] Loss_t (custom k) = 0.5357372761        
[1 ] Loss_t = 1.118463516           Loss_k = -21.15161514           Loss = -10.01657581        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1744728982        
[0] Loss_t (custom k) = 0.457197547         
[1 ] Loss_t = 0.8538625836          Loss_k = -21.16059494           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.04275377095       
[0] Loss_t (custom k) = 0.4885528684        
[1 ] Loss_t = 1.053029776           Loss_k = -21.45175743           Loss = -10.19936371        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.07623396069       
[0] Loss_t (custom k) = 0.5802995563        
[1 ] Loss_t = 1.343928099           Loss_k = -21.45878983           Loss = -10.05743122        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1805866659        
[0] Loss_t (custom k) = 0.6219348311        
[1 ] Loss_t = 0.9494707584          Loss_k = -21.46106339           Loss = -10.25579643        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1867413819        
[0] Loss_t (custom k) = 0.5361973047        
[1 ] Loss_t = 1.129894972           Loss_k = -21.47441101           Loss = -10.17225838        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1225008965        
[0] Loss_t (custom k) = 0.5891020894        
[1 ] Loss_t = 1.951896667           Loss_k = -21.47563362           Los

In [11]:
# Adv after 2 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.5712976


In [12]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.05019379407       
[0] Loss_t (custom k) = 0.4156490266        
[1 ] Loss_t = 1.12534833            Loss_k = -21.55042267           Loss = -10.21253681        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.08819077164       
[0] Loss_t (custom k) = 0.3886458576        
[1 ] Loss_t = 0.8949867487          Loss_k = -21.55884743           Loss = -10.33193016        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1419844031        
[0] Loss_t (custom k) = 0.5051864982        
[1 ] Loss_t = 0.6403122544          Loss_k = -21.56709099           Loss = -10.4633894         
Returned k: [ 64   4 193 128]
[0] Loss_t = 0.2370337248        
[0] Loss_t (custom k) = 0.8570222259        
[1 ] Loss_t = 1.573281288           Loss_k = -21.5439415            Loss = -9.985330582        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.2232460678        
[0] Loss_t (custom k) = 0.5425593853        
[1 ] Loss_t = 1.148141146           Loss_k = -21.58238411           Los

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1824121773        
[0] Loss_t (custom k) = 0.490432471         
[1 ] Loss_t = 1.247738719           Loss_k = -21.83085632           Loss = -10.29155922        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.154166311         
[0] Loss_t (custom k) = 0.5944890976        
[1 ] Loss_t = 1.473010302           Loss_k = -21.83757782           Loss = -10.1822834         
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1150390357        
[0] Loss_t (custom k) = 0.3829343319        
[1 ] Loss_t = 0.9532404542          Loss_k = -21.84903526           Loss = -10.44789696        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2117867172        
[0] Loss_t (custom k) = 0.7234801054        
[1 ] Loss_t = 2.132094383           Loss_k = -21.85451889           Loss = -9.861211777        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.147298187         
[0] Loss_t (custom k) = 0.6491444707        
[1 ] Loss_t = 1.432781696           Loss_k = -21.85672951           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1936954558        
[0] Loss_t (custom k) = 0.6994796991        
[1 ] Loss_t = 1.90695703            Loss_k = -22.08358765           Loss = -10.08831501        
Returned k: [ 64   4 197 128]
[0] Loss_t = 0.1181766689        
[0] Loss_t (custom k) = 0.6271260977        
[1 ] Loss_t = 0.9521891475          Loss_k = -22.074049             Loss = -10.56093025        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1755669564        
[0] Loss_t (custom k) = 0.5598931313        
[1 ] Loss_t = 1.231748104           Loss_k = -22.09534264           Loss = -10.43179703        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1320533007        
[0] Loss_t (custom k) = 0.5918931961        
[1 ] Loss_t = 1.522198439           Loss_k = -22.09608078           Loss = -10.28694153        
Returned k: [ 64   4 197 128]
[0] Loss_t = 0.05797974765       
[0] Loss_t (custom k) = 0.5313856602        
[1 ] Loss_t = 1.371095181           Loss_k = -22.09152985           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1116945669        
[0] Loss_t (custom k) = 0.4635518491        
[1 ] Loss_t = 1.594461679           Loss_k = -22.30764961           Loss = -10.35659409        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1613087058        
[0] Loss_t (custom k) = 0.4558310211        
[1 ] Loss_t = 0.6658443213          Loss_k = -22.31364441           Loss = -10.82390022        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1760733426        
[0] Loss_t (custom k) = 0.6082619429        
[1 ] Loss_t = 1.020107985           Loss_k = -22.31825256           Loss = -10.64907265        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1917080879        
[0] Loss_t (custom k) = 0.6535028815        
[1 ] Loss_t = 1.752773762           Loss_k = -22.32389641           Loss = -10.28556156        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1610500216        
[0] Loss_t (custom k) = 0.5577352047        
[1 ] Loss_t = 0.6204389334          Loss_k = -22.3295784            Los

In [13]:
# Adv after 3 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.56591016


In [ ]:
history = []

for e in range(10):
    print(f'Epoch {e+1}')
    for batch, (x, y) in enumerate(train_dataset):
        loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
                adv_model, adv_optimizer, epochs=50, random_k=True)
        
    metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
    print(metrics.result().numpy())
    history.append(metrics)

Epoch 1
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2653899491        
[0] Loss_t (custom k) = 0.5548875928        
[1 ] Loss_t = 1.258549333           Loss_k = -25.09499931           Loss = -11.91822529        
[2 ] Loss_t = 1.248559117           Loss_k = -25.09682274           Loss = -11.92413139        
[3 ] Loss_t = 1.23507309            Loss_k = -25.0986557            Loss = -11.93179131        
[4 ] Loss_t = 1.216681719           Loss_k = -25.10049629           Loss = -11.94190693        
[5 ] Loss_t = 1.192151785           Loss_k = -25.10234261           Loss = -11.95509529        
[6 ] Loss_t = 1.160925627           Loss_k = -25.10419464           Loss = -11.97163486        
[7 ] Loss_t = 1.124003768           Loss_k = -25.10605049           Loss = -11.99102306        
[8 ] Loss_t = 1.080486178           Loss_k = -25.10791016           Loss = -12.01371193        
[9 ] Loss_t = 1.029881239           Loss_k = -25.10977173           Loss = -12.0399456         
[10] Loss_t = 0.973

[34] Loss_t = 0.2387645394          Loss_k = -25.25885391           Loss = -12.51004505        
[35] Loss_t = 0.2280232757          Loss_k = -25.26086617           Loss = -12.51642132        
[36] Loss_t = 0.217111215           Loss_k = -25.26287651           Loss = -12.52288246        
[37] Loss_t = 0.2060394436          Loss_k = -25.26488495           Loss = -12.52942276        
[38] Loss_t = 0.1947400123          Loss_k = -25.26689148           Loss = -12.53607559        
[39] Loss_t = 0.1831204593          Loss_k = -25.2688961            Loss = -12.54288769        
[40] Loss_t = 0.1712834388          Loss_k = -25.27089882           Loss = -12.54980755        
[41] Loss_t = 0.1591668874          Loss_k = -25.27290154           Loss = -12.5568676         
[42] Loss_t = 0.1463950127          Loss_k = -25.27490234           Loss = -12.56425381        
[43] Loss_t = 0.1340455115          Loss_k = -25.27690506           Loss = -12.57143021        
[44] Loss_t = 0.1231839806          Loss

[18] Loss_t = 0.728801012           Loss_k = -25.40173531           Loss = -12.33646679        
[19] Loss_t = 0.6904599667          Loss_k = -25.40356445           Loss = -12.35655212        
[20] Loss_t = 0.6528379917          Loss_k = -25.40539169           Loss = -12.37627697        
[21] Loss_t = 0.6160978079          Loss_k = -25.40721512           Loss = -12.39555836        
[22] Loss_t = 0.5807086825          Loss_k = -25.40903473           Loss = -12.41416264        
[23] Loss_t = 0.5466349125          Loss_k = -25.41084862           Loss = -12.43210697        
[24] Loss_t = 0.5089883804          Loss_k = -25.41265488           Loss = -12.45183372        
[25] Loss_t = 0.4698879421          Loss_k = -25.4144516            Loss = -12.47228146        
[26] Loss_t = 0.4375020266          Loss_k = -25.41623688           Loss = -12.48936749        
[27] Loss_t = 0.4140408039          Loss_k = -25.41802216           Loss = -12.50199032        
[28] Loss_t = 0.3914245963          Loss

[2 ] Loss_t = 1.701185584           Loss_k = -25.56571007           Loss = -11.93226242        
[3 ] Loss_t = 1.637808323           Loss_k = -25.56754303           Loss = -11.96486759        
[4 ] Loss_t = 1.552279472           Loss_k = -25.56935883           Loss = -12.0085392         
[5 ] Loss_t = 1.464549065           Loss_k = -25.57116127           Loss = -12.05330658        
[6 ] Loss_t = 1.389348865           Loss_k = -25.57295609           Loss = -12.09180355        
[7 ] Loss_t = 1.323692918           Loss_k = -25.57474899           Loss = -12.12552834        
[8 ] Loss_t = 1.265916109           Loss_k = -25.57654381           Loss = -12.15531349        
[9 ] Loss_t = 1.212771177           Loss_k = -25.57834053           Loss = -12.18278503        
[10] Loss_t = 1.159951806           Loss_k = -25.58013916           Loss = -12.2100935         
[11] Loss_t = 1.109668016           Loss_k = -25.58193779           Loss = -12.23613453        
[12] Loss_t = 1.05867362            Loss

[38] Loss_t = 0.2739241123          Loss_k = -25.69742584           Loss = -12.71175098        
[39] Loss_t = 0.2645881772          Loss_k = -25.69915199           Loss = -12.7172823         
[40] Loss_t = 0.2561132908          Loss_k = -25.70088005           Loss = -12.7223835         
[41] Loss_t = 0.2483448088          Loss_k = -25.70261002           Loss = -12.7271328         
[42] Loss_t = 0.2412135005          Loss_k = -25.70434189           Loss = -12.73156452        
[43] Loss_t = 0.2345560789          Loss_k = -25.70607567           Loss = -12.73575974        
[44] Loss_t = 0.2282930464          Loss_k = -25.70780945           Loss = -12.73975849        
[45] Loss_t = 0.222356528           Loss_k = -25.70954704           Loss = -12.74359512        
[46] Loss_t = 0.216768235           Loss_k = -25.71128464           Loss = -12.74725819        
[47] Loss_t = 0.2114750147          Loss_k = -25.71302223           Loss = -12.75077343        
[48] Loss_t = 0.2064458281          Loss

...

[30] Loss_t = 1.180437326           Loss_k = -33.24639511           Loss = -16.03297806        
[31] Loss_t = 1.169962645           Loss_k = -33.24648666           Loss = -16.03826141        
[32] Loss_t = 1.164847851           Loss_k = -33.24658203           Loss = -16.04086685        
[33] Loss_t = 1.160208702           Loss_k = -33.24667358           Loss = -16.04323196        
[34] Loss_t = 1.154783726           Loss_k = -33.24676514           Loss = -16.04598999        
[35] Loss_t = 1.148737907           Loss_k = -33.24685669           Loss = -16.04905891        
[36] Loss_t = 1.142825603           Loss_k = -33.24695206           Loss = -16.05206299        
[37] Loss_t = 1.138360977           Loss_k = -33.24704742           Loss = -16.05434418        
[38] Loss_t = 1.133703828           Loss_k = -33.24714279           Loss = -16.05671883        
[39] Loss_t = 1.130289555           Loss_k = -33.24723434           Loss = -16.05847168        
[40] Loss_t = 1.123932123           Loss

[14] Loss_t = 1.37042439            Loss_k = -33.25922012           Loss = -15.94439793        
[15] Loss_t = 1.350039244           Loss_k = -33.25931931           Loss = -15.95464039        
[16] Loss_t = 1.340388536           Loss_k = -33.25941849           Loss = -15.95951462        
[17] Loss_t = 1.327465057           Loss_k = -33.25951767           Loss = -15.96602631        
[18] Loss_t = 1.314589739           Loss_k = -33.25961685           Loss = -15.9725132         
[19] Loss_t = 1.301177502           Loss_k = -33.25971985           Loss = -15.97927094        
[20] Loss_t = 1.290404797           Loss_k = -33.25981903           Loss = -15.98470688        
[21] Loss_t = 1.279368877           Loss_k = -33.25991821           Loss = -15.99027443        
[22] Loss_t = 1.266412973           Loss_k = -33.26002121           Loss = -15.99680424        
[23] Loss_t = 1.243855596           Loss_k = -33.26012039           Loss = -16.00813293        
[24] Loss_t = 1.226113915           Loss

[50] Loss_t = 1.690677881           Loss_k = -33.26751709           Loss = -15.78841972        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2160746455        
[0] Loss_t (custom k) = 0.6373937726        
[1 ] Loss_t = 1.64051199            Loss_k = -33.27319717           Loss = -15.81634235        
[2 ] Loss_t = 1.623288751           Loss_k = -33.2732811            Loss = -15.82499599        
[3 ] Loss_t = 1.613592625           Loss_k = -33.27336884           Loss = -15.82988834        
[4 ] Loss_t = 1.598169565           Loss_k = -33.27345276           Loss = -15.83764172        
[5 ] Loss_t = 1.581835032           Loss_k = -33.27353668           Loss = -15.84585094        
[6 ] Loss_t = 1.565142632           Loss_k = -33.27362442           Loss = -15.85424042        
[7 ] Loss_t = 1.551846027           Loss_k = -33.27370834           Loss = -15.8609314         
[8 ] Loss_t = 1.541487455           Loss_k = -33.27379227           Loss = -15.86615276        
[9 ] Loss_t = 1.530811548  

[34] Loss_t = 1.152225256           Loss_k = -33.2696228            Loss = -16.05869865        
[35] Loss_t = 1.146362305           Loss_k = -33.26971817           Loss = -16.06167793        
[36] Loss_t = 1.139918923           Loss_k = -33.26981354           Loss = -16.06494713        
[37] Loss_t = 1.132629275           Loss_k = -33.26990891           Loss = -16.06863976        
[38] Loss_t = 1.124844193           Loss_k = -33.27000809           Loss = -16.07258224        
[39] Loss_t = 1.106914997           Loss_k = -33.27010345           Loss = -16.08159447        
[40] Loss_t = 1.088806033           Loss_k = -33.27019882           Loss = -16.09069633        
[41] Loss_t = 1.084374428           Loss_k = -33.27029419           Loss = -16.09296036        
[42] Loss_t = 1.081379175           Loss_k = -33.27038956           Loss = -16.09450531        
[43] Loss_t = 1.078388214           Loss_k = -33.27048492           Loss = -16.09604836        
[44] Loss_t = 1.07191062            Loss

[18] Loss_t = 1.300443292           Loss_k = -33.2778244            Loss = -15.98869038        
[19] Loss_t = 1.29102838            Loss_k = -33.27791977           Loss = -15.9934454         
[20] Loss_t = 1.285540104           Loss_k = -33.27801132           Loss = -15.99623585        
[21] Loss_t = 1.279624343           Loss_k = -33.27810287           Loss = -15.99923897        
[22] Loss_t = 1.275049925           Loss_k = -33.27819824           Loss = -16.00157356        
[23] Loss_t = 1.270789862           Loss_k = -33.27828979           Loss = -16.00374985        
[24] Loss_t = 1.267481565           Loss_k = -33.27837753           Loss = -16.00544739        
[25] Loss_t = 1.264564037           Loss_k = -33.27846909           Loss = -16.00695229        
[26] Loss_t = 1.260490775           Loss_k = -33.27856064           Loss = -16.00903511        
[27] Loss_t = 1.257485747           Loss_k = -33.27865219           Loss = -16.01058388        
[28] Loss_t = 1.254333973           Loss

In [28]:
def visualize():
    x, y = next(iter(train_dataset))

    sentence = x[0].numpy().tolist()
    sentence = sentence[0:sentence.index(0)]
    print(sentence)
    sentence_str = encoder.decode(x[0])    
    print(sentence_str)
    tokens = [encoder.decode([w]) for w in sentence]
    
    print(len(tokens))
    sent_original, _ = sentiment(sentence_str, encoder, transformer)
    sent, weights = sentiment(sentence_str,    encoder=encoder, 
                      adv_model=adv_model, transformer=transformer)
    # sent, weights = sentiment(sentence, encoder=encoder, transformer=old_transformer)

    print(f"Initial sentiment: {sent_original}")
    print(f'Adv sentiment: {sent}')
    head_view(torch.tensor(weights.numpy()[0]), tokens)

In [ ]:
visualize()